In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [15]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


In [23]:
latent_dim = 1

encoder_inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 14, 14, 32)   320         input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 7, 7, 64)     18496       conv2d_6[0][0]                   
__________________________________________________________________________________________________
flatten_3 (Flatten)             (None, 3136)         0           conv2d_7[0][0]                   
____________________________________________________________________________________________

In [24]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
dense_5 (Dense)              (None, 3136)              6272      
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 14, 14, 64)        36928     
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 28, 28, 1)         289       
Total params: 61,953
Trainable params: 61,953
Non-trainable params: 0
_______________________________________________________

In [25]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


In [34]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
train_idx = np.any([y_train==9, y_train==4], axis=0)
test_idx = np.any([y_test==9, y_test==4], axis=0)
x_train = x_train[train_idx]
y_train = y_train[train_idx]
x_test = x_test[test_idx]
y_test = y_test[test_idx]
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
history = vae.fit(mnist_digits, epochs=100, batch_size=128)


Epoch 1/100
108/108 [==============================] - 19s 165ms/step - loss: 146.7416 - reconstruction_loss: 143.2248 - kl_loss: 3.4466
Epoch 2/100
108/108 [==============================] - 19s 173ms/step - loss: 145.9307 - reconstruction_loss: 142.9554 - kl_loss: 3.4332
Epoch 3/100
108/108 [==============================] - 18s 171ms/step - loss: 146.5853 - reconstruction_loss: 142.9022 - kl_loss: 3.4137
Epoch 4/100
108/108 [==============================] - 18s 171ms/step - loss: 146.2310 - reconstruction_loss: 142.8891 - kl_loss: 3.4034
Epoch 5/100
108/108 [==============================] - 18s 171ms/step - loss: 146.0680 - reconstruction_loss: 142.8618 - kl_loss: 3.3849
Epoch 6/100
108/108 [==============================] - 18s 170ms/step - loss: 146.3552 - reconstruction_loss: 142.8882 - kl_loss: 3.3602
Epoch 7/100
108/108 [==============================] - 19s 172ms/step - loss: 146.1236 - reconstruction_loss: 142.7911 - kl_loss: 3.3493
Epoch 8/100
108/108 [====================

108/108 [==============================] - 8s 75ms/step - loss: 144.8658 - reconstruction_loss: 141.8045 - kl_loss: 3.0263
Epoch 57/100
108/108 [==============================] - 8s 77ms/step - loss: 145.2876 - reconstruction_loss: 141.8276 - kl_loss: 3.0171
Epoch 58/100
108/108 [==============================] - 8s 75ms/step - loss: 145.9484 - reconstruction_loss: 141.8286 - kl_loss: 3.0175
Epoch 59/100
108/108 [==============================] - 8s 75ms/step - loss: 144.8677 - reconstruction_loss: 141.8204 - kl_loss: 3.0196: 0s - loss: 144.8753 - reconstruction_loss: 141.7960 - kl_l
Epoch 60/100
108/108 [==============================] - 8s 76ms/step - loss: 144.7188 - reconstruction_loss: 141.7881 - kl_loss: 3.0272: 0s - loss: 144.7154 - reconstruction_loss: 141.8715 - kl_loss: 3.02
Epoch 61/100
108/108 [==============================] - 8s 75ms/step - loss: 144.6735 - reconstruction_loss: 141.7993 - kl_loss: 3.0290
Epoch 62/100
108/108 [==============================] - 8s 75ms/step

In [33]:
import matplotlib.pyplot as plt


def plot_latent_space(vae, n=50, figsize=15):
    # display a n*n 2D manifold of digits
    digit_size = 28
    scale = 5.
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]

    for i, yi in enumerate(grid_y):
#         for j, xi in enumerate(grid_x):
        for j, xi in [(0,1)]:
            z_sample = np.array([[yi]])
            x_decoded = vae.decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[
                i * digit_size : (i + 1) * digit_size,
                j * digit_size : (j + 1) * digit_size,
            ] = digit

    plt.figure(figsize=(figsize, figsize))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.show()


plot_latent_space(vae)